In [1]:
from model import *
import random
import nltk

def show_result(input_sentence,output_tag):
   
    assert(len(input_sentence) == len(output_tag))
    for i in range(len(input_sentence)):
        w,t = input_sentence[i],output_tag[i]
        if t[0]=="B":
            print("[#{} {} ".format(t[2:],w),end="")
            if (i+1>=len(output_tag) or output_tag[i+1][0]!="I"):
                print("]",end=" ")
        elif t[0]=="I" and (i+1>=len(output_tag) or output_tag[i+1][0]!="I"):
            print(w,"]",end=" ")
        else:
            print(w,end=" ")
    print()
            

In [2]:

# Data parameters
task = 'ner'  
train_file = 'datasets/train.txt'  
val_file = 'datasets/dev.txt'  
test_file = 'datasets/test.txt' 

min_word_freq = 5 
min_char_freq = 1  
caseless = False  
expand_vocab = True  

# Model parameters
char_emb_dim = 30 
word_emb_dim = 100
word_rnn_dim = 300  
char_rnn_dim = 300  
char_rnn_layers = 1  
word_rnn_layers = 1 
highway_layers = 1  
dropout = 0.5  
fine_tune_word_embeddings = True  

# Training parameters
start_epoch = 0  
batch_size = 10 
lr = 0.015  
lr_decay = 0.05  #
momentum = 0.9  
workers = 1  
epochs = 200 
grad_clip = 5.  
print_freq = 100  
best_f1 = 0.  
checkpoint =  'BESTner.pth.tar'  

epochs_since_improvement = 0
tag_ind = 3   # choose column in  dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wsg_checkpoint = "BESTwsg.pth.tar"

In [3]:
wsg_state = torch.load(wsg_checkpoint, map_location=lambda storage, loc: storage)
wsg_model = wsg_state['model']
wsg_word_map = wsg_state['word_map']
wsg_lm_vocab_size = wsg_state['lm_vocab_size']
wsg_tag_map = wsg_state['tag_map']
wsg_char_map = wsg_state['char_map']

def wsg(words):

    global wsg_word_map, wsg_char_map, wsg_tag_map

   
    
    #print(tag_map)
    #print(char_map)
    
    
    # Loss functions
    lm_criterion = nn.CrossEntropyLoss().to(device)
    crf_criterion = ViterbiLoss(tag_map).to(device)

   
    vb_decoder = ViterbiDecoder(wsg_tag_map)
    
    temp_word_map = {k: v for k, v in wsg_word_map.items() if v <= wsg_word_map['<unk>']}
    
    
    tags = [["O"]*len(words[i]) for i in range(len(words))]
    
    val_inputs = create_input_tensors(words, tags, temp_word_map, wsg_char_map, wsg_tag_map)
    val_loader = torch.utils.data.DataLoader(WCDataset(*val_inputs), batch_size=batch_size, shuffle=True,
                                             num_workers=workers, pin_memory=False)
                   

    wsg_model.eval()

    batch_time = AverageMeter()
    vb_losses = AverageMeter()
    
    start = time.time()
    output = []
    for i, (wmaps, cmaps_f, cmaps_b, cmarkers_f, cmarkers_b, tmaps, wmap_lengths, cmap_lengths) in enumerate(
            val_loader):

        max_word_len = max(wmap_lengths.tolist())
        max_char_len = max(cmap_lengths.tolist())

        wmaps = wmaps[:, :max_word_len].to(device)
        cmaps_f = cmaps_f[:, :max_char_len].to(device)
        cmaps_b = cmaps_b[:, :max_char_len].to(device)
        cmarkers_f = cmarkers_f[:, :max_word_len].to(device)
        cmarkers_b = cmarkers_b[:, :max_word_len].to(device)
        tmaps = tmaps[:, :max_word_len].to(device)
        wmap_lengths = wmap_lengths.to(device)
        cmap_lengths = cmap_lengths.to(device)

        # Forward prop.
        crf_scores, wmaps_sorted, tmaps_sorted, wmap_lengths_sorted, _, __ = wsg_model(cmaps_f,
                                                                                   cmaps_b,
                                                                                   cmarkers_f,
                                                                                   cmarkers_b,
                                                                                   wmaps,
                                                                                   tmaps,
                                                                                   wmap_lengths,
                                                                                   cmap_lengths)

       
        decoded = vb_decoder.decode(crf_scores.to("cpu"), wmap_lengths_sorted.to("cpu"))
        decoded, _ = pack_padded_sequence(decoded, (wmap_lengths_sorted - 1).tolist(), batch_first=True)
       
        inv={v:k for k,v in wsg_tag_map.items()}
        w = []
        for j,t in enumerate([inv[k] for k in decoded.numpy()]):
            if t=="O" or t=="B":
                w.append(words[i][j])
            else:
                if len(w)==0:
                    w.append(words[i][j])
                else:
                    w[-1]+="_"+words[i][j]
        output.append(w)
    return output
        

In [4]:
state = torch.load(checkpoint, map_location=lambda storage, loc: storage)
model = state['model']
optimizer = state['optimizer']
word_map = state['word_map']
lm_vocab_size = state['lm_vocab_size']
tag_map = state['tag_map']
char_map = state['char_map']
start_epoch = state['epoch'] + 1
best_f1 = state['f1']

val_words, val_tags = read_words_tags(val_file, 3, caseless)


def ner(words):

    global best_f1, epochs_since_improvement, state, start_epoch, word_map, char_map, tag_map

   
    
    #print(tag_map)
    #print(char_map)
    
    
    # Loss functions
    lm_criterion = nn.CrossEntropyLoss().to(device)
    crf_criterion = ViterbiLoss(tag_map).to(device)

   
    vb_decoder = ViterbiDecoder(tag_map)
    
    temp_word_map = {k: v for k, v in word_map.items() if v <= word_map['<unk>']}
    
    
    #print("len: ",len(val_words))

    
    tags = [["O"]*len(words[0])]
   
    
    val_inputs = create_input_tensors(words, tags, temp_word_map, char_map, tag_map)
    val_loader = torch.utils.data.DataLoader(WCDataset(*val_inputs), batch_size=batch_size, shuffle=True,
                                             num_workers=workers, pin_memory=False)
                   

    model.eval()

    batch_time = AverageMeter()
    vb_losses = AverageMeter()
    f1s = AverageMeter()

    start = time.time()

    for i, (wmaps, cmaps_f, cmaps_b, cmarkers_f, cmarkers_b, tmaps, wmap_lengths, cmap_lengths) in enumerate(
            val_loader):

        max_word_len = max(wmap_lengths.tolist())
        max_char_len = max(cmap_lengths.tolist())

        wmaps = wmaps[:, :max_word_len].to(device)
        cmaps_f = cmaps_f[:, :max_char_len].to(device)
        cmaps_b = cmaps_b[:, :max_char_len].to(device)
        cmarkers_f = cmarkers_f[:, :max_word_len].to(device)
        cmarkers_b = cmarkers_b[:, :max_word_len].to(device)
        tmaps = tmaps[:, :max_word_len].to(device)
        wmap_lengths = wmap_lengths.to(device)
        cmap_lengths = cmap_lengths.to(device)

        # Forward prop.
        crf_scores, wmaps_sorted, tmaps_sorted, wmap_lengths_sorted, _, __ = model(cmaps_f,
                                                                                   cmaps_b,
                                                                                   cmarkers_f,
                                                                                   cmarkers_b,
                                                                                   wmaps,
                                                                                   tmaps,
                                                                                   wmap_lengths,
                                                                                   cmap_lengths)

       
        decoded = vb_decoder.decode(crf_scores.to("cpu"), wmap_lengths_sorted.to("cpu"))
        decoded, _ = pack_padded_sequence(decoded, (wmap_lengths_sorted - 1).tolist(), batch_first=True)
       
        inv={v:k for k,v in tag_map.items()}
        #print([inv[k] for k in decoded.numpy()])
        
        print("INPUT:")
        print()
        print(" ".join(words[0]))
        print()
        
        output = [inv[k] for k in decoded.numpy()]
        show_result(words[0],output)
        
        print()
        

# Chạy một câu ngẫu nhiên từ tập test

In [57]:
index = random.randrange(0,len(val_words)-1)
words = [val_words[index]]
ner(words)
true_tags = [val_tags[index]]
show_result(words[0],true_tags[0])

INPUT:

Khi đến được nơi người kia nằm toán lính Mỹ nhận thấy người đó đang bảo_vệ các bệnh_nhân trong một bệnh_viện .

Khi đến được nơi người kia nằm toán lính [#LOC Mỹ ] nhận thấy người đó đang bảo_vệ các bệnh_nhân trong một bệnh_viện . 

Khi đến được nơi người kia nằm toán lính [#LOC Mỹ ] nhận thấy người đó đang bảo_vệ các bệnh_nhân trong một bệnh_viện . 


# chạy một câu từ dữ liệu nhập vào

In [58]:
sent = """
GS Nguyễn Minh Thuyết - Tổng chủ biên Chương trình Giáo dục phổ thông cũng bày tỏ với báo chí: “Việc công nhận tiếng Anh là ngôn ngữ thứ hai của Việt Nam trước sau cũng phải thực hiện. Công nhận tiếng Anh như ngôn ngữ thứ hai sẽ thúc đẩy việc lớp trẻ học tiếng Anh. Nếu không phải toàn dân học tiếng Anh thì ít nhất là lớp trẻ”
"""
token = nltk.word_tokenize(sent)
print(token)
word_segment = wsg([token])
print()
print(word_segment)
print()
ner(word_segment)

['GS', 'Nguyễn', 'Minh', 'Thuyết', '-', 'Tổng', 'chủ', 'biên', 'Chương', 'trình', 'Giáo', 'dục', 'phổ', 'thông', 'cũng', 'bày', 'tỏ', 'với', 'báo', 'chí', ':', '“', 'Việc', 'công', 'nhận', 'tiếng', 'Anh', 'là', 'ngôn', 'ngữ', 'thứ', 'hai', 'của', 'Việt', 'Nam', 'trước', 'sau', 'cũng', 'phải', 'thực', 'hiện', '.', 'Công', 'nhận', 'tiếng', 'Anh', 'như', 'ngôn', 'ngữ', 'thứ', 'hai', 'sẽ', 'thúc', 'đẩy', 'việc', 'lớp', 'trẻ', 'học', 'tiếng', 'Anh', '.', 'Nếu', 'không', 'phải', 'toàn', 'dân', 'học', 'tiếng', 'Anh', 'thì', 'ít', 'nhất', 'là', 'lớp', 'trẻ', '”']

[['GS', 'Nguyễn', 'Minh', 'Thuyết', '-', 'Tổng', 'chủ_biên', 'Chương_trình', 'Giáo_dục', 'phổ_thông', 'cũng', 'bày_tỏ', 'với', 'báo_chí', ':', '“', 'Việc', 'công_nhận', 'tiếng', 'Anh', 'là', 'ngôn_ngữ', 'thứ', 'hai', 'của', 'Việt_Nam', 'trước', 'sau', 'cũng', 'phải', 'thực_hiện', '.', 'Công_nhận', 'tiếng', 'Anh', 'như', 'ngôn_ngữ', 'thứ', 'hai', 'sẽ', 'thúc_đẩy', 'việc', 'lớp', 'trẻ', 'học', 'tiếng', 'Anh', '.', 'Nếu', 'không', 'phải